# Continue from intermediate results

## Overview

It is a common need to be able to continue the pipeline from some intermediate result computed earlier.



## Setup

Lets look at a situation where we have some "raw" data files and the workflow consists of three steps
  * loading the raw data
  * cleaning the raw data
  * computing a sum of the cleaned data.

In [ ]:
from typing import NewType

Filename = NewType('Filename', str)
RawData = NewType('RawData', list)
CleanData = NewType('CleanData', list)
Result = NewType('Result', list)

filesystem = {'raw.txt': list(map(str, range(10)))}

def load(filename: Filename) -> RawData:
    """Load the data from the filename."""
    data = filesystem[filename]
    return RawData(data)

def clean(raw_data: RawData) -> CleanData:
    """Clean the data, convert from str."""
    return CleanData(list(map(float, raw_data)))

def process(clean_data: CleanData) -> Result:
    """Compute the sum of the clean data."""
    return Result(sum(clean_data))


In [ ]:
import sciline

pipeline = sciline.Pipeline(
    [load, clean, process,],
    params={ Filename: 'raw.txt', })
pipeline

## Setting intermediate results

If we select `Result` the task graph will use the `Filename` input because it needs to read the raw data from the file system:

In [ ]:
pipeline.get(Result)

But if the cleaned data has already been produced it is unnecessary to "re-clean" it, in that case we can proceed directly from the clean data to the compute sum step.
To do this we replace the `CleanData` provider with the data that was loaded and cleaned:

In [ ]:
data = pipeline.compute(CleanData)
pipeline[CleanData] = data
pipeline

Then if we select `Result` the task graph will no longer use the `Filename` input and instead it will proceed directly from the `CleanData` as input:

In [ ]:
pipeline.get(Result)

In [ ]:
pipeline.compute(Result)